In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.naive_bayes import MultinomialNB
import random
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

262 documents
19 classes ['about', 'canswer1', 'canswer2', 'canswer3', 'comments', 'complaint', 'fanswer1', 'fanswer2', 'feedback', 'goodbye', 'greeting', 'help', 'name', 'negative', 'neutral', 'noanswer', 'options', 'postive', 'thanks']
296 unique lemmatized words ["'m", "'s", ',', '.', 'a', 'about', 'affordable', 'again', 'alright', 'always', 'am', 'amazing', 'an', 'and', 'angry', 'annoyed', 'anyone', 'are', 'assist', 'at', 'atmosphere', 'attentive', 'authentic', 'awesome', 'awful', 'bad', 'been', 'best', 'better', 'beyond', 'bitterful', 'blowing', 'by', 'bye', 'call', 'calling', 'can', 'care', 'cash', 'choice', 'close', 'cold', 'complain', 'complaint', 'consistently', 'content', 'cooked', 'could', 'courteous/happy', 'cuisine', 'customer', 'cya', 'day', 'decent', 'definitely', 'delicous', 'delighted', 'delivered', 'delivery', 'demanded', 'didnt', 'displeaded', 'displeased', 'dissatified', 'dissatisfied', 'do', 'done', 'dry', 'duty', 'enjoyed', 'even', 'ever', 'exceeded', 'excellent',

In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\admin\anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py:22: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model BLEU  
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/100
53/53 [==============================] - 1s 1ms/step - loss: 2.9110 - accuracy: 0.1076
Epoch 2/100
53/53 [==============================] - 0s 1ms/step - loss: 2.6463 - accuracy: 0.1728
Epoch 3/100
53/53 [==============================] - 0s 1ms/step - loss: 2.5797 - accuracy: 0.2126
Epoch 4/100
53/53 [==============================] - 0s 2ms/step - loss: 2.3979 - accuracy: 0.3044
Epoch 5/100
53/53 [==============================] - 0s 2ms/step - loss: 2.1812 - accuracy: 0.3449
Epoch 6/100
53/53 [==============================] - 0s 2ms/step - loss: 2.1459 - accuracy: 0.3065
Epoch 7/100
53/53 [==============================] - 0s 1ms/step - loss: 1.8842 - accuracy: 0.4320
Epoch 8/100
53/53 [==============================] - 0s 2ms/step - loss: 1.7959 - accuracy: 0.4398
Epoch 9/100
53/53 [==============================] - 0s 1ms/step - loss: 1.7097 - accuracy: 0.4971
Epoch 10/100
53/53 [==============================] - 0s 2ms/step - loss: 1.5446 - accuracy: 0.5525
Epoch 11/

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [8]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res


In [9]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        with open('op1.txt', 'a+') as f:
            f.write('\nuser: {}\nBot: {}\n'.format(msg,res))
        with open('op2.txt', 'a+') as f:
            f.write('\n{}\n'.format(msg))
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
    
base = Tk()
base.title("Foodie_Bot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

In [10]:
data = pd.read_csv('op2.txt', header = None)
print (data)

                          0
0               Users_input
1                     Hello
2                 Yourself?
3                  feedback
4                     Hello
5                        hi
6               how are you
7           food is not bad
8              food is good
9                     Hello
10                      Hai
11  Happy with your service
12                    hello


In [11]:
import pandas as pd
import numpy as np
df = pd.read_csv('Restaurant_Reviews.csv', sep=',') 
df.head()

,Review,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Review     1000 non-null   object
 1   sentiment  1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [13]:
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string
punctuations = string.punctuation
parser = English()
stopwords = list(STOP_WORDS)
def spacy_tokenizer(utterance):
    tokens = parser(utterance)
    return [token.lemma_.lower().strip() for token in tokens if token.text.lower().strip() not in stopwords and token.text not in punctuations]

In [14]:
# Preprocessing
nltk.download('stopwords')
corpus = []
for i in range(0, 1000): #as the data as 1000 data points
    review = re.sub('[^a-zA-Z]', ' ', df['Review'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Evaluating model

In [15]:
from sklearn.model_selection import train_test_split
reviews = df['Review'].values
labels = df['sentiment'].values
reviews_train, reviews_test, y_train, y_test = train_test_split(reviews, labels,test_size = 0.3, random_state = 42)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
#By default, the vectorizer might be created as follows:
#vectorizer = CountVectorizer()
vectorizer.fit(reviews_train)
# Creating the Bag of Words model
cv = CountVectorizer(max_features = 2000)
#the X and y
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:, 1].values
#X- data
#y- labels

C:\Users\admin\anaconda3\envs\notebook\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



In [17]:
X_train = vectorizer.transform(reviews_train)
X_test = vectorizer.transform(reviews_test)

In [18]:
from sklearn import svm
clf=svm.SVC(kernel='linear').fit(X_train,y_train)

In [19]:
classifier_prediction=clf.predict(X_test)
print(accuracy_score(y_test,classifier_prediction)*100)

49.333333333333336


In [20]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=0.1)
classifier.fit(X_train, y_train)

MultinomialNB(alpha=0.1)

In [21]:
predicted = classifier.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted,y_test)

In [22]:
accuracy = classifier.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.49333333333333335


In [23]:
print(str('{:04.2f}'.format(accuracy_score*100))+'%')

49.33%


In [24]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression() 

In [25]:
#Fit model on data
lr.fit(X_train,y_train)

LogisticRegression()

In [26]:
accuracy = classifier.score(X_test, y_test)
print("Accuracy:", accuracy)
print(str('{:04.2f}'.format(accuracy_score*100))+'%')

Accuracy: 0.49333333333333335
49.33%


In [27]:
#Make predictions
predictions = lr.predict(X_test)

In [28]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 501,  criterion = 'entropy')                             
model.fit(X_train, y_train) 

RandomForestClassifier(criterion='entropy', n_estimators=501)

In [29]:
y_pred = model.predict(X_test)
y_pred

array([0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1], d

In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.49      0.47      0.48       152
           1       0.47      0.49      0.48       148

    accuracy                           0.48       300
   macro avg       0.48      0.48      0.48       300
weighted avg       0.48      0.48      0.48       300



In [31]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(accuracy_score(y_test, y_pred)*100)

48.0


In [32]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[71, 81],
       [75, 73]], dtype=int64)

In [33]:
#Testing find accuracy, precision, recall:
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)

array([[  0,   0],
       [152, 148]], dtype=int64)

In [34]:
#Predictions on New Data
new_reviews =pd.read_csv('op2.txt',sep='\t')
X_new = vectorizer.transform(new_reviews)
classifier.predict(X_new)

array([1], dtype=int64)